In [1]:
import numpy as np
from IPython.display import HTML, display

In [2]:
A = np.array([[1, 1, 1],
              [0, 1, 0]])

B = np.array([[1, 1, 1],
              [0, 1, 0]])

C = np.array([[1, 1, 0],
              [0, 1, 1]])

D = np.array([[1, 1, 1],
              [0, 0, 1],
              [0, 0, 1]])

E = np.array([[1, 1, 0],
              [0, 1, 0],
              [0, 1, 1]])

In [21]:

def ones(np_array):
    indices = np.where(np_array.flatten() == 1)[0]
    return tuple(indices)


class PuzzleSolver:
    def __init__(self, pieces, width=6, height=4):
        self.color_map = {
            0: 'black',
            1: 'red',
            2: 'green',
            3: 'yellow',
            4: 'blue',
            5: 'magenta',
            6: 'white'
        }

        self.blank_board = np.zeros((height, width))
        self.size = height * width
        self.width = width
        self.height = height
        self.pieces = pieces
        self.num_pieces = len(pieces)
        self.oriented_pieces = []

        def embed(piece):
            embedded_piece = self.blank_board.copy()
            embedded_piece[:piece.shape[0], :piece.shape[1]] = piece
            return embedded_piece

        for piece in pieces:
            orientations = set()
            orientations.add(ones(embed(piece)))
            orientations.add(ones(embed(np.rot90(piece, k=1))))
            orientations.add(ones(embed(np.rot90(piece, k=2))))
            orientations.add(ones(embed(np.rot90(piece, k=3))))
            orientations.add(ones(embed(piece.transpose())))
            orientations.add(ones(embed(np.rot90(piece.transpose(), k=1))))
            orientations.add(ones(embed(np.rot90(piece.transpose(), k=2))))
            orientations.add(ones(embed(np.rot90(piece.transpose(), k=3))))
            self.oriented_pieces.append(np.array(list(orientations)))

    def print_colored_matrix(self, matrix):
        reshaped_matrix = matrix.reshape((self.height, self.width))
        html_output = "<div style='font-family:monospace;'>"

        for row in reshaped_matrix:
            html_output += "<div>"
            for val in row:
                color = self.color_map[val]
                html_output += f"<span style='color:{color};'>██</span>"
            html_output += "</div>"

        html_output += "</div>"
        display(HTML(html_output))

    def print_colored_matrix_with_box(self, matrix):
        reshaped_matrix = matrix.reshape((self.height, self.width))
        html_output = "<div style='font-family:monospace;'>"

        # Top border
        html_output += "<div>" + "<span style='color:white;'>██</span>" * (self.width + 2) + "</div>"

        for row in reshaped_matrix:
            html_output += "<div>"
            html_output += "<span style='color:white;'>██</span>"  # Left border
            for val in row:
                color = self.color_map[val]
                html_output += f"<span style='color:{color};'>██</span>"
            html_output += "<span style='color:white;'>██</span>"  # Right border
            html_output += "</div>"

        # Bottom border
        html_output += "<div>" + "<span style='color:white;'>██</span>" + "<span style='color:black;'>██</span>"*2 + "<span style='color:white;'>██</span>"*(self.width-1) + "</div>"

        html_output += "</div><br>"
        display(HTML(html_output))

    def valid(self, ind):
        if max(ind) >= self.size:
            return False
        rightmost = max(ind % self.width)
        leftmost = min(ind % self.width)
        return rightmost - leftmost <= 2

    def indicator(self, ind):
        board = self.blank_board.flatten().copy()
        board[ind] += 1
        return board.copy()

    def make_matrices(self):
        self.matrices = []
        num_pieces = len(self.oriented_pieces)

        for m in range(num_pieces):
            mini_matrix = self.oriented_pieces[m]
            matrix = (m+1) * np.array([
                self.indicator(mini_matrix[j, :] + i)
                for j in range(mini_matrix.shape[0])
                for i in range(self.size)
                if self.valid(mini_matrix[j, :] + i)
            ])
            self.matrices.append(matrix)

    def zip_up(self, A_mat, B_mat, same=False):
        AB_prod = np.einsum('ji,ki->jk', A_mat, B_mat)
        AB_list = []

        for i in range(AB_prod.shape[0]):
            j_range = i if same else AB_prod.shape[1]
            for j in range(j_range):
                if AB_prod[i, j] == 0:
                    AB_list.append(A_mat[i, :] + B_mat[j, :])

        return np.array(AB_list)

    def find_arrangements(self):
        A_mat, B_mat, C_mat, D_mat, E_mat = self.matrices
        AB_mat = self.zip_up(A_mat, B_mat, same=True)
        CD_mat = self.zip_up(C_mat, D_mat)
        CDE_mat = self.zip_up(CD_mat, E_mat)
        self.final_mat = self.zip_up(AB_mat, CDE_mat)

    def show_arrangements(self, box=False):

        self.num_arrangements = self.final_mat.shape[0]
        print('Displaying all possible final configurations ('+str(self.num_arrangements)+ ').')
        for i in range(self.num_arrangements):
            print(i+1)
            if not box:
                self.print_colored_matrix(self.final_mat[i, :])
            else:
                self.print_colored_matrix_with_box(self.final_mat[i, :])


    def can_slide(self, flat_board , avoid=[]):

        board = flat_board.reshape((self.height, self.width))
        for k in range(1, self.num_pieces+1):
          isolation = (board==k)
          if isolation.sum()==0:
            pass
          else:
            rest = board - k * isolation


            big_board = np.ones((self.height+2, self.width+2))
            big_board[1:-1,1:-1]*=0



            e_rest = big_board.copy()
            e_rest[1:-1,1:-1] = rest
            e_rest[-1,1:3] *= 0

            #print(e_rest.reshape((self.height+2, self.width+2)))
            #self.print_colored_matrix(e_rest[1:-1,1:-1])

            e_isolation0 = 0 * big_board.copy()
            e_isolation0[1:-1,1:-1] = isolation

            e_isolation1 = 0 * big_board.copy()
            e_isolation1[2:,1:-1] = isolation

            e_isolation2 = 0 * big_board.copy()
            e_isolation2[1:-1,2:] = isolation

            e_isolation3 = 0 * big_board.copy()
            e_isolation3[0:-2,1:-1] = isolation

            e_isolation4 = 0 * big_board.copy()
            e_isolation4[1:-1,0:-2] = isolation



            if (e_isolation1 * e_rest).sum()==0 and not (k, 'down') in avoid:
              return (True, e_rest[1:-1,1:-1].flatten(), 'down', k, e_isolation1[1:-1,1:-1].flatten())
            if (e_isolation2 * e_rest).sum()==0 and not (k, 'right') in avoid:
              return (True, e_rest[1:-1,1:-1].flatten(),'right', k, e_isolation2[1:-1,1:-1].flatten())
            if (e_isolation3 * e_rest).sum()==0 and not (k, 'up') in avoid:
              return (True, e_rest[1:-1,1:-1].flatten(), 'up', k, e_isolation3[1:-1,1:-1].flatten())
            if (e_isolation4 * e_rest).sum()==0 and not (k, 'left') in avoid:
              return (True, e_rest[1:-1,1:-1].flatten(), 'left', k, e_isolation4[1:-1,1:-1].flatten())

        return (False, None)

    def check_slideyness(self):
        print()
        print()
        print('Trying to find slidey solutions...')
        print()
        total=0
        for i in range(self.final_mat.shape[0]):

          slidey = self.can_slide(self.final_mat[i, :])
          if slidey[0]:
            shifted_piece = slidey[4]
            new_board = slidey[1] #+shifted_piece

            new_slidey = self.can_slide(new_board, avoid=[(slidey[3], slidey[2])])
            if new_slidey[0]:
              print('Can you start by sliding the', self.color_map[slidey[3]],'piece', slidey[2] + ' in configuration number '+str(i)+ '?')
              self.print_colored_matrix_with_box(self.final_mat[i, :] )
              total+=1

        self.slidey_possibilities = total

    def print_summary(self):
        print('Total number of arrangements: '+ str(self.num_arrangements) + '.')
        try:
          print('Narrowed down to', self.slidey_possibilities, 'sliding options.')
        except:
          pass

In [22]:

mySolver = PuzzleSolver([A, B, C, D, E], width=6, height=4)
mySolver.make_matrices()
mySolver.find_arrangements()
mySolver.show_arrangements(box=True)
mySolver.check_slideyness()
mySolver.print_summary()

Displaying all possible final configurations (52).
1


2


3


4


5


...

52




Trying to find slidey solutions...

Can you start by sliding the yellow piece down in configuration number 4?


Can you start by sliding the yellow piece down in configuration number 23?


Can you start by sliding the magenta piece down in configuration number 37?


Total number of arrangements: 52.
Narrowed down to 3 sliding options.
